## Hello linear model
This tutorial is part of the Tensorflow getting started documentation at: https://www.tensorflow.org/tutorials/wide

In [2]:
import tensorflow as tf
import os
import shutil
import sys

model_dir='./tmp/model'
train_data='./sample-data/rotor-tap-sample-data.csv'

# delete the model directory
shutil.rmtree(model_dir, ignore_errors=True)

# declare feature columns within csv
median_gyro_roll = tf.feature_column.numeric_column(key='median-gyro-roll', dtype=tf.float64);
median_gyro_pitch = tf.feature_column.numeric_column(key='median-gyro-pitch', dtype=tf.float64);
median_gyro_yaw = tf.feature_column.numeric_column(key='median-gyro-yaw', dtype=tf.float64);

median_acc_x = tf.feature_column.numeric_column(key='median-acc-x', dtype=tf.float64);
median_acc_y = tf.feature_column.numeric_column(key='median-acc-y', dtype=tf.float64);
median_acc_z = tf.feature_column.numeric_column(key='median-acc-z', dtype=tf.float64);

mean_gyro_roll = tf.feature_column.numeric_column(key='mean-gyro-roll', dtype=tf.float64);
mean_gyro_pitch = tf.feature_column.numeric_column(key='mean-gyro-pitch', dtype=tf.float64);
mean_gyro_yaw = tf.feature_column.numeric_column(key='mean-gyro-yaw', dtype=tf.float64);

# stack feature columns into a single array
imu_window_feature_columns = [median_gyro_roll, median_gyro_pitch, median_gyro_yaw,
        median_acc_x, median_acc_y, median_acc_z,
        mean_gyro_roll, mean_gyro_pitch, mean_gyro_yaw]

run_config=tf.estimator.RunConfig().replace(
    session_config=tf.ConfigProto(device_count={'GPU': 0})
)

def input_fn(data_file):
    assert tf.gfile.Exists(data_file),('%s not found')
    records_default = [['0'], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0],
                       [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
    csv_columns = [
                    'rotor',
                    'median-gyro-roll','median-gyro-pitch','median-gyro-yaw',
                    'median-acc-x','median-acc-y','median-acc-z',
                    'mean-gyro-roll','mean-gyro-pitch','mean-gyro-yaw',
                    'mean-acc-x','mean-acc-y','mean-acc-z',
                    'skew-gyro-roll','skew-gyro-pitch','skew-gyro-yaw',
                    'skew-acc-x','skew-acc-y','skew-acc-z'
                  ]
    
    def parse_csv(value):
        print('PARSING:', data_file)
        columns = tf.decode_csv(value, records_default)
        features = dict(zip(csv_columns, columns))
        labels = features.pop('rotor')
        print('LABELS:', labels)
        return features, labels
    
    dataset = tf.data.TextLineDataset(data_file)
    dataset = dataset.shuffle(200)
    dataset = dataset.map(parse_csv, 4)
    dataset = dataset.batch(200)
    return dataset

# model = tf.estimator.LinearClassifier(
#     model_dir=model_dir,
#     feature_columns=columns,
#     config=run_config,
#     n_classes=4,
#     label_vocabulary=['1', '2', '3', '4']
# )
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    feature_columns=imu_window_feature_columns,
    config=run_config,
    hidden_units=[100, 75, 50, 25],
    n_classes=4,
    label_vocabulary=['1', '2', '3', '4']
)



# Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
for n in range(40 // 2):
    model.train(input_fn=lambda: input_fn(
        train_data))

    results = model.evaluate(input_fn=lambda: input_fn(
        train_data))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * 2)
    print('-' * 60)





# model.train(input_fn=lambda: input_fn(
#     train_data))

# results = model.evaluate(input_fn=lambda:input_fn(
#     train_data
# ))

INFO:tensorflow:Using config: {'_session_config': device_count {
  key: "GPU"
}
, '_task_type': 'worker', '_model_dir': './tmp/model', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_master': '', '_num_ps_replicas': 0, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000274ED0328D0>, '_save_checkpoints_secs': 600}
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/model\model.ckpt.
INFO:tensorflow:step = 1, loss = 3380.2302
INFO:tensorflow:Loss for final step: 3380.2302.
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Startin

Results at epoch 18
------------------------------------------------------------
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/model\model.ckpt-9
INFO:tensorflow:Saving checkpoints for 10 into ./tmp/model\model.ckpt.
INFO:tensorflow:step = 10, loss = 285.515
INFO:tensorflow:Loss for final step: 285.515.
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Starting evaluation at 2018-02-18-01:49:49
INFO:tensorflow:Restoring parameters from ./tmp/model\model.ckpt-10
INFO:tensorflow:Finished evaluation at 2018-02-18-01:49:50
INFO:tensorflow:Saving dict for global step 10: accuracy = 0.6875, average_loss = 6.4991236, global_step = 10, loss = 103.98598
Results at epoch 20
------------------------------------------------------------
PARSING: ./sample-data/rotor-tap-sample-data.csv

INFO:tensorflow:step = 19, loss = 12.259648
INFO:tensorflow:Loss for final step: 12.259648.
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Starting evaluation at 2018-02-18-01:50:47
INFO:tensorflow:Restoring parameters from ./tmp/model\model.ckpt-19
INFO:tensorflow:Finished evaluation at 2018-02-18-01:50:48
INFO:tensorflow:Saving dict for global step 19: accuracy = 0.875, average_loss = 0.46744472, global_step = 19, loss = 7.4791155
Results at epoch 38
------------------------------------------------------------
PARSING: ./sample-data/rotor-tap-sample-data.csv
LABELS: Tensor("DecodeCSV:0", shape=(), dtype=string)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/model\model.ckpt-19
INFO:tensorflow:Saving checkpoints for 20 into ./tmp/model\model.ckpt.
INFO:tensorflow:step = 20, loss = 7.479115
INFO:tensorflow:Loss for final step: 7.479115.
PARSING: ./sample-data/rotor-tap-s